In [1]:
import sys
import argparse
import random
import openslide
import numpy as np
import PIL.Image as Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.models as models
import utility_functions as uf
import pandas as pd
import os
import shutil
import xlwt
from xlwt import Workbook
import xlrd
import utility_functions as uf

In [131]:
#checking the data
index = 2000
lib = torch.load("/home/weber50432/AML_image_processing/lib/NPM1_patch_20000/NPM1_patch_20000_test_data.pt")
# lib = torch.load("/home/weber50432/AML_image_processing/lib/NPM1_patch_500_balanced/NPM1_patch_500_balanced_test_data.pt")
slidename = lib["slides"]
print(lib["slides"][:5])
print(type(lib["grid"][:5]))
# print(lib["grid"][:5][0])
print(lib["targets"][:5])
# print(len(lib["slides"]))
grid = []
slideIDX = []
count = 0
for i,g in enumerate(lib['grid']):
  grid.extend(g)
  slideIDX.extend([i]*len(g))
print(slideIDX[0])
print(type(grid))
print(grid[0])
slide_num = slidename[1].split("_")[0]
slide_path = f"/staging/biology/b08611005/ROI_level0_pixel512/{slide_num}/"
print(slide_path)
patch_path = slide_path + grid[index]
print(patch_path)

['/staging/biology/b08611005/ROI_level0_pixel512/A29', '/staging/biology/b08611005/ROI_level0_pixel512/A60', '/staging/biology/b08611005/ROI_level0_pixel512/A99', '/staging/biology/b08611005/ROI_level0_pixel512/A102', '/staging/biology/b08611005/ROI_level0_pixel512/A106']
<class 'list'>
[0, 0, 1, 0, 0]
0
<class 'list'>
A29_32389.png
/staging/biology/b08611005/ROI_level0_pixel512//staging/biology/b08611005/ROI/
/staging/biology/b08611005/ROI_level0_pixel512//staging/biology/b08611005/ROI/A29_40665.png


In [14]:
#checking the data
index = random.randint(0, 10000)
# lib = torch.load("/home/weber50432/AML_image_processing/lib/NPM1_patch_20000/NPM1_patch_20000_test_data.pt")
lib = torch.load("/home/weber50432/AML_image_processing/lib/NPM1_patch_500_upsampled/NPM1_patch_500_upsampled_test_data.pt")
slidename = lib["slides"]
print(lib["slides"][:5])
# print(lib["grid"][:5])
# print(lib["targets"][:5])
grid = []
slideIDX = []
count = 0
for i,g in enumerate(lib['grid']):
  grid.extend(g)
  slideIDX.extend([i]*len(g))
# print(slideIDX[index])
# print(grid[index])
slide_num = slidename[slideIDX[index]].split("_")[0]
slide_path = f"/staging/biology/b08611005/ROI_level0_pixel512/{slide_num}/"
print(slide_path)
patch_path = slide_path + grid[index]
print(patch_path)

['A844', 'A109_1', 'A367_9', 'A467_5', 'A994_3']
/staging/biology/b08611005/ROI_level0_pixel512/A234/
/staging/biology/b08611005/ROI_level0_pixel512/A234/A234_56545.png


In [4]:
# test the get_grid function
y = uf.get_grid("/home/exon_storage1/aml_slide/",[413],256,0.6,0.15,500)
print(len(y[0]))
print(y[0][:10])

slide 413 done, slect 500 patches from 1356 total patches.
500
[(97280, 26368), (110080, 77056), (94720, 31232), (98048, 79104), (112128, 69888), (118272, 31744), (112640, 72192), (95744, 23040), (113152, 73216), (95488, 35072)]


In [9]:
# test the get_patches_grid function
path = "/home/weber50432/AML_image_processing/HCT_region_select/output/"
slide_list = os.listdir(path)
# remove the "A" in each element in the list
slide_list = [int(i[1:]) for i in slide_list]
slide_list.sort()
print(slide_list)
print(len(slide_list))
# grid = uf.get_patches_grid(path,slide_list,1)
# print(grid)

[3, 9, 12, 13, 22, 26, 27, 28, 29, 60, 99, 102, 103, 104, 105, 106, 108, 109, 113, 121, 123, 129, 130, 131, 133, 134, 135, 137, 138, 140, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 156, 158, 159, 160, 161, 162, 163, 165, 166, 167, 168, 169, 171, 172, 173, 174, 176, 177, 178, 180, 197, 199, 200, 201, 203, 204, 205, 206, 207, 208, 209, 212, 217, 219, 220, 227, 230, 231, 232, 233, 234, 235, 238, 239, 241, 242, 244, 265, 273, 274, 280, 299, 314, 317, 319, 320, 322, 323, 330, 331, 332, 333, 334, 335, 336, 337, 338, 341, 342, 343, 344, 345, 347, 348, 349, 350, 352, 353, 355, 357, 359, 360, 361, 363, 364, 365, 366, 367, 369, 370, 371, 372, 373, 374, 377, 378, 380, 381, 382, 383, 386, 387, 390, 392, 393, 395, 397, 1001]
148


In [11]:
# replace the // with / in excel file in the output folder
excel_path = "/home/weber50432/AML_image_processing/HCT_region_select/output/A13/porduction_result.xls"
if os.path.exists(excel_path):
  wb = xlrd.open_workbook(excel_path)
  sheet1 = wb.sheet_by_index(0)
  sheet2 = wb.sheet_by_index(1)
  pos_list = sheet1.col_values(0)
  neg_list = sheet2.col_values(0)
  pos_list.pop(0)
  neg_list.pop(0)
  wb = Workbook()
  sheetNumber0 = wb.add_sheet('Sheet 0')
  sheetNumber1 = wb.add_sheet('Sheet 1')

  row = 0
  sheetNumber0.write(row, 0, 'Posivitve List')
  sheetNumber0.write(row, 1, len(pos_list))
  row += 1
  for i in pos_list:
    new_string = i.replace("//", "/")
    sheetNumber0.write(row, 0, new_string)
    row += 1

  row = 0
  sheetNumber1.write(row, 0, 'Negative List')
  sheetNumber1.write(row, 1, len(neg_list))
  row += 1
  for i in neg_list:
    new_string = i.replace("//", "/")
    sheetNumber1.write(row, 0, new_string)
    row += 1
  wb.save(excel_path)

In [6]:
# check best epoch in .pth file
loaded_obj = torch.load("/home/weber50432/AML_image_processing/MIL_slide_level/output/NPM1_K50/checkpoint_best.pth")
print(loaded_obj['epoch'])


50


In [5]:
libraryfile = "/home/weber50432/AML_image_processing/lib/NPM1_patch_20000/NPM1_patch_20000_train_data.pt"
lib = torch.load(libraryfile)
print('loading', libraryfile)
#Flatten grid
grid = []
slideIDX = []
for i,g in enumerate(lib['grid']):
    grid.extend(g)
    slideIDX.extend([i]*len(g))
    print(len(g))
print('Number of tiles: {}'.format(len(grid)))
slidenames = lib['slides']
targets = lib['targets']
grid = grid
slideIDX = slideIDX
mode = None
mult = lib['mult']
size = int(np.round(224*lib['mult']))
level = lib['level']
index = 500
patch_path = slidenames[slideIDX[index]] +"/"+ grid[index]
print(size)
print(patch_path)

loading /home/weber50432/AML_image_processing/lib/NPM1_patch_20000/NPM1_patch_20000_train_data.pt
2147
2468
2837
4080
2265
2067
1476
6032
12243
8064
6239
2232
1051
6605
9433
2425
4527
5914
3431
4824
3402
4267
2424
6162
2256
5062
2572
8620
4543
1557
5873
5157
1697
5110
6103
2447
3511
4259
7511
3888
957
2169
5053
4683
1209
6114
5730
3345
1684
2403
2544
992
305
483
1597
3720
1140
4023
2069
1301
3987
4516
3004
3398
1533
5177
3676
2814
5662
1656
2040
1874
2793
3033
4861
3066
3922
8837
1222
2053
213
1500
2563
3912
5243
5193
3654
3035
2252
2628
3724
775
3953
2845
5386
2171
6006
2256
746
1603
4087
5671
5480
2294
2093
380
1339
5404
5303
5206
11160
6380
2564
2201
5749
444
7481
6821
6760
1513
3302
2710
3104
5878
1833
3208
2582
3300
1865
3559
5826
4373
3404
3313
2612
2669
8632
4758
5638
3586
986
1564
2127
4955
5456
3707
6258
2676
6215
1366
2440
2636
2375
2361
145
7644
2152
1976
5134
4142
2526
1480
3504
6050
8835
762
1153
3963
2825
2067
2345
3956
5595
2084
1377
4316
3712
1221
3994
1101
3641
8952
49

In [8]:
checked_list = os.listdir("/home/weber50432/AML_image_processing/HCT_cell_detection/output")
print(len(checked_list))
for i in os.listdir("/home/exon_storage1/aml_slide/ROI_level0_pixel512/"):
  if i not in checked_list:
    print(i)

572
A1027
A1122
